# Description

In this demo, we use the same data as previous and do different types of visualisations. 


# Clone a data repository

As we will use the data for several weeks, I decided to create a separate repository for the data. We can clone the reposiotory to Colab working directory to start working on the data.

There is parliamentary constituency shape file, downloaded from here:

https://geoportal.statistics.gov.uk/datasets/5ce27b980ffb43c39b012c2ebeab92c0_2


In [ ]:
!rm -r GV918-UK-politics-data
!git clone  https://github.com/University-of-Essex-Dept-of-Government/GV918-UK-politics-data

# Load packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

## Load the data

In [ ]:
df_merge = pd.read_csv("/content/GV918-UK-politics-data/Data/merged_brexit_data.csv")

# Heatmap

Heatmat is one of the good tool for telling a story with data. 

For example, the figure, included in the slides used by the scientific advisor at the press conference for second lockdown announcement, conveys a rich information about the general tendency across the regions and age groups (see page 5).

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/931774/Slides_to_accompany_coronavirus_press_conference-_CMO-_31_October_2020.pdf

We will create a simple heatmap from the correlation table:

In [ ]:
tab_cor = df_merge.loc[:, ['con_pct', 'lab_pct','leave_pct', 'Age_percent', 'ConstPercentChangeOneYr', 'UnempConstRate']].corr()
tab_cor

In [ ]:
#df_merge.ConstPercentChangeOneYr.astype(float, errors = 'coerse')

pd.to_numeric(df_merge.ConstPercentChangeOneYr, errors='coerce')

In [ ]:
sns.heatmap(tab_cor)

In [ ]:
plt.figure(figsize = (7, 7))
sns.heatmap(tab_cor)

In [ ]:
sns.heatmap(tab_cor, center = 0)

In [ ]:
sns.heatmap(tab_cor, center = 0, cmap = 'coolwarm')

In [ ]:
np.fill_diagonal(tab_cor.values, 0)

In [ ]:
tab_cor

In [ ]:
sns.heatmap(tab_cor, center = 0, cmap = 'coolwarm')

In [ ]:
sns.heatmap(tab_cor, center = 0, cmap = 'coolwarm', vmin = -1, vmax = 1)

# Plotting the values on the map

Here, we will see how to create a figure using a map. Whether you can craete a map figure largely depends on the availability of shapfiles, which describes the shapes of polygons for the geographical boudaries. Once you find the file, it is not too complicated.

First, we install a package to work with shapefiles in pandas/matplotlib.

### Install geopandas

In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd

### Unzip shapefile

In [ ]:
!unzip /content/GV918-UK-politics-data/Data/Westminster_Parliamentary_Constituencies__December_2017__Boundaries_UK-shp.zip \
 -d /content/const-shape

In [ ]:
!ls /content/const-shape

### Read the shapefile

In [ ]:
df_map = gpd.read_file('/content/const-shape/Westminster_Parliamentary_Constituencies__December_2017__Boundaries_UK.shp')

In [ ]:
df_map.head()

## Plot map

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (8, 8))
df_map.plot(ax = ax,edgecolor='none').set_axis_off()

### Merge political dataset



In [ ]:
df_polmap = df_map.merge(df_merge, left_on = 'pcon17cd', right_on = 'ons_id')

### Two maps: UK and London Separately

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (8, 8))
df_polmap.plot(ax = axes[0], edgecolor = 'none').set_axis_off()

df_polmap[df_polmap['region_name'] == "London"].plot(ax = axes[1], edgecolor = 'none').set_axis_off()

### Plot Leave Percentage

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (8, 8))
df_polmap.plot(ax = ax,edgecolor='none', column = 'leave_pct', legend = True).set_axis_off()


### Plot leave percentage (UK and London, two maps)

In [ ]:
fig, axes = plt.subplots(1,2, figsize = (12, 7))
df_polmap.plot(ax = axes[0],edgecolor='none', column = 'leave_pct', legend = True).set_axis_off()
df_polmap[df_polmap['region_name'] == "London"].plot(ax = axes[1],edgecolor='none', column = 'leave_pct', legend = True).set_axis_off()


### Final figure

In [ ]:
pal = 'Reds'
vmin = 0.2
vmax = 0.8

fig, axes = plt.subplots(1,2, figsize = (12, 8))

df_polmap.plot(ax = axes[0],edgecolor='none', column = 'leave_pct', 
               cmap = pal, vmin = vmin, vmax = vmax).set_axis_off()

df_polmap[df_polmap['region_name'] == "London"].plot(
    ax = axes[1], edgecolor='none', column = 'leave_pct',
    cmap = pal, vmin = vmin, vmax = vmax).set_axis_off()

sm = plt.cm.ScalarMappable(cmap=pal, 
                           norm=plt.Normalize(vmin=vmin, 
                           vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm)


### Categorical (winning parties)

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (8, 8))
df_polmap.plot(ax = ax,edgecolor='none', column = 'first_party', legend = True,
               cmap = 'Set1').set_axis_off()

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (8, 8))
df_polmap[df_polmap['first_party'].isin(['Con', 'Lab', 'LD', 'SNP', 'PC', 'DUP', 'SF'])].plot(
    ax = ax,edgecolor='none', column = 'first_party', legend = True,
               cmap = 'Set1').set_axis_off()